In [1]:
import glob
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import rocks

rocks.set_log_level("error")
import time as t


from astropy.time import Time
import requests
import io

import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import angular_separation

# from scipy.signal import argrelextrema

from scipy.stats import ks_2samp
import ssptools

from figure_mask import compute_mask, print_statistics, compute_mask_spin

In [2]:
import figure_setup as fs

# Get ZTF fit

In [3]:
# Local Configuration
data_fink = "../"
bft_file = os.path.join(data_fink, "data", "ssoBFT-latest.parquet")

In [4]:
# ZTF filters 1: g, 2: r
filters = {"1": "g", "2": "r"}

S_color = {"g": -0.3928, "r": 0.2913}
sun_color = {"g": -0.3044, "r": 0.1903}

color_C = -(sun_color["g"] - sun_color["r"])
color_S = -(S_color["g"] - S_color["r"])

fink_colors = ["#15284F", "#F5622E"]

V_minus_g = -0.2833
V_minus_r = 0.1777

In [5]:
data = pd.read_parquet(os.path.join(data_fink, "data", "ztf", "sso_ZTF.parquet"))

In [6]:
data["SHG1G2_dSpin"] = np.sqrt(
    (data["SHG1G2_dalpha0"] * np.cos(np.radians(data["SHG1G2_delta0"]))) ** 2
    + data["SHG1G2_ddelta0"] ** 2
)


# # Remove solutions above 90 deg of latitude
cond = data.SHG1G2_delta0 > 90
data.loc[cond, "SHG1G2_delta0"] = 90
print(f"above 90: {len(data[cond])} ")

cond = data.SHG1G2_delta0 < -90
data.loc[cond, "SHG1G2_delta0"] = -90
print(f"below 90: {len(data[cond])} ")

above 90: 0 
below 90: 0 


In [7]:
cols = [
    "sso_number",
    "sso_name",
    "sso_class",
    "orbital_elements.semi_major_axis.value",
    "orbital_elements.eccentricity.value",
    "orbital_elements.inclination.value",
    "orbital_elements.node_longitude.value",
    "orbital_elements.perihelion_argument.value",
    "orbital_elements.mean_anomaly.value",
    "orbital_elements.mean_motion.value",
    "family.family_number",
    "family.family_name",
    "proper_elements.proper_semi_major_axis.value",
    "proper_elements.proper_eccentricity.value",
    "proper_elements.proper_inclination.value",
    "proper_elements.proper_sine_inclination.value",
    "tisserand_parameters.Jupiter.value",
    "albedo.value",
    "absolute_magnitude.value",
    "diameter.value",
    "taxonomy.class",
    "taxonomy.complex",
    "taxonomy.waverange",
    "taxonomy.scheme",
    "taxonomy.technique",
    "colors.g-r.color.value",
    "colors.g-r.color.error.min",
    "colors.g-r.color.error.max",
    "colors.g-r.facility",
    "colors.g-r.observer",
    "colors.g-r.epoch",
    "colors.g-r.delta_time",
    "colors.g-r.id_filter_1",
    "colors.g-r.id_filter_2",
    "colors.g-r.phot_sys",
    "colors.g-r.technique",
    "colors.g-i.color.value",
    "colors.g-i.color.error.min",
    "colors.g-i.color.error.max",
    "colors.g-i.facility",
    "colors.g-i.observer",
    "colors.g-i.epoch",
    "colors.g-i.delta_time",
    "colors.g-i.id_filter_1",
    "colors.g-i.id_filter_2",
    "colors.g-i.phot_sys",
    "colors.g-i.technique",
    "colors.i-z.color.value",
    "colors.i-z.color.error.min",
    "colors.i-z.color.error.max",
    "colors.i-z.facility",
    "colors.i-z.observer",
    "colors.i-z.epoch",
    "colors.i-z.delta_time",
    "colors.i-z.id_filter_1",
    "colors.i-z.id_filter_2",
    "colors.i-z.phot_sys",
    "colors.i-z.technique",
    "spins.1.obliquity",
    "spins.1.RA0.value",
    "spins.1.DEC0.value",
    "spins.1.RA0.error.max",
    "spins.1.DEC0.error.max",
    "spins.1.long.value",
    "spins.1.lat.value",
    "spins.1.technique",
    "spins.2.obliquity",
    "spins.2.RA0.value",
    "spins.2.DEC0.value",
    "spins.2.RA0.error.max",
    "spins.2.DEC0.error.max",
    "spins.2.long.value",
    "spins.2.lat.value",
    "spins.2.technique",
    "spins.3.obliquity",
    "spins.3.RA0.value",
    "spins.3.DEC0.value",
    "spins.3.RA0.error.max",
    "spins.3.DEC0.error.max",
    "spins.3.long.value",
    "spins.3.lat.value",
    "spins.3.technique",
    "spins.4.obliquity",
    "spins.4.RA0.value",
    "spins.4.DEC0.value",
    "spins.4.RA0.error.max",
    "spins.4.DEC0.error.max",
    "spins.4.long.value",
    "spins.4.lat.value",
    "spins.4.technique",
]
bft = pd.read_parquet(bft_file, columns=cols)

In [8]:
data = data.merge(bft[cols], left_on="name", right_on="sso_name", how="left")

In [9]:
# data

In [10]:
mask_SHG1G2_union = compute_mask(data, model='SHG1G2', kind='union')
mask_HG1G2_union = compute_mask(data, model='HG1G2', kind='union')
mask_HG_union = compute_mask(data, model='HG', kind='union')

mask_SHG1G2_inter = compute_mask(data, model='SHG1G2', kind='inter')
mask_HG1G2_inter = compute_mask(data, model='HG1G2', kind='inter')
mask_HG_inter = compute_mask(data, model='HG', kind='inter')

maskFINK_inter = compute_mask(data, R_min=0.305, model='FINK', kind='inter')
maskFINK_union = compute_mask(data, R_min=0.305, model='FINK', kind='union')

for model in ['HG', 'HG1G2', 'SHG1G2', 'FINK']:
    print_statistics(data, model)
    print()

  All data       : 122675  (100.00%)
  Mask HG (g∩r) : 120010  ( 97.83%)
  Mask HG (gUr) : 120011  ( 97.83%)

  All data       : 122675  (100.00%)
  Mask HG1G2 (g∩r) :  47175  ( 38.46%)
  Mask HG1G2 (gUr) :  85366  ( 69.59%)

  All data       : 122675  (100.00%)
  Mask SHG1G2 (g∩r) :  64209  ( 52.34%)
  Mask SHG1G2 (gUr) :  97549  ( 79.52%)

  All data       : 122675  (100.00%)
  Mask FINK (g∩r) :  50163  ( 40.89%)
  Mask FINK (gUr) :  73284  ( 59.74%)



In [11]:
maskFINK = maskFINK_union

# General description of the sample

In [12]:
data.loc[maskFINK, "sso_class"].value_counts()

sso_class
MB>Middle       23988
MB>Inner        22506
MB>Outer        19373
Phocaea           859
Hungaria          842
Trojan            707
Mars-Crosser      697
MB>Cybele         339
MB>Hilda          244
NEA>Apollo         75
NEA>Amor           72
NEA>Aten            9
Centaur             2
NEA>Atira           2
KBO>SDO             1
Name: count, dtype: int64

In [13]:
As = data.loc[maskFINK, "orbital_elements.semi_major_axis.value"]
Es = data.loc[maskFINK, "orbital_elements.eccentricity.value"]
node = data.loc[maskFINK, "orbital_elements.inclination.value"]

fig, ax = plt.subplots(figsize=(12, 8))

ax.scatter(As, np.sin(np.radians(node)), marker=".", alpha=0.2, s=2)


ax.set_xlim(1.5, 5.5)
ax.set_ylim(0, 0.6)
ax.set_xlabel("Semi-major axis (au)")
ax.set_ylabel(r"Sine of inclination ($\sin$i)")

# fig.savefig(f'{data_fink}plots/types.png', facecolor='white', dpi=150)

Text(0, 0.5, 'Sine of inclination ($\\sin$i)')

# Spins!

In [14]:
data.columns[data.columns.str.contains("alpha0")]

Index(['SHG1G2_alpha0', 'SHG1G2_dalpha0', 'SHG1G2_alpha0_alt'], dtype='object')

In [15]:
# Create alternative spins solutions
xax = "SHG1G2_alpha0"
yax = "SHG1G2_delta0"

# Already done at Fink level
# data["SHG1G2_alpha0_alt"] = (data["SHG1G2_alpha0"] + 180) % 360
# data["SHG1G2_delta0_alt"] = -data["SHG1G2_delta0"]


data["SHG1G2_alpha0_rand"] = data["SHG1G2_alpha0"]
data["SHG1G2_delta0_rand"] = data["SHG1G2_delta0"]
cond = (data.index % 2) == 0
data.loc[cond, "SHG1G2_alpha0_rand"] = data.loc[cond, "SHG1G2_alpha0_alt"]
data.loc[cond, "SHG1G2_delta0_rand"] = data.loc[cond, "SHG1G2_delta0_alt"]


# Convert to EC
coords = SkyCoord(
    ra=data["SHG1G2_alpha0"].values * u.deg,
    dec=data["SHG1G2_delta0"].values * u.deg,
    distance=200 * u.parsec,
    frame="hcrs",
)
data["lon"] = coords.heliocentricmeanecliptic.lon.value
data["lat"] = coords.heliocentricmeanecliptic.lat.value

coords = SkyCoord(
    ra=data["SHG1G2_alpha0_alt"].values * u.deg,
    dec=data["SHG1G2_delta0_alt"].values * u.deg,
    distance=200 * u.parsec,
    frame="hcrs",
)
data["lon_alt"] = coords.heliocentricmeanecliptic.lon.value
data["lat_alt"] = coords.heliocentricmeanecliptic.lat.value


coords = SkyCoord(
    ra=data["SHG1G2_alpha0_rand"].values * u.deg,
    dec=data["SHG1G2_delta0_rand"].values * u.deg,
    distance=200 * u.parsec,
    frame="hcrs",
)
data["lon_rand"] = coords.heliocentricmeanecliptic.lon.value
data["lat_rand"] = coords.heliocentricmeanecliptic.lat.value

In [16]:
# Obliquity of the spin
data["lon_orbit"] = data["orbital_elements.node_longitude.value"] - 90
data["lat_orbit"] = 90.0 - data["orbital_elements.inclination.value"]
data["obliquity"] = data[["lon", "lat", "lon_orbit", "lat_orbit"]].apply(
    lambda x: np.degrees(
        angular_separation(
            np.radians(x[0]), np.radians(x[1]), np.radians(x[2]), np.radians(x[3])
        )
    ),
    axis=1,
)
data["obliquity_alt"] = data[["lon_alt", "lat_alt", "lon_orbit", "lat_orbit"]].apply(
    lambda x: np.degrees(
        angular_separation(
            np.radians(x[0]), np.radians(x[1]), np.radians(x[2]), np.radians(x[3])
        )
    ),
    axis=1,
)

/tmp/ipykernel_3519359/3937172300.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  np.radians(x[0]), np.radians(x[1]), np.radians(x[2]), np.radians(x[3])
/tmp/ipykernel_3519359/3937172300.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  np.radians(x[0]), np.radians(x[1]), np.radians(x[2]), np.radians(x[3])


# Figure longitude vs inclination

In [17]:
Is = np.sin(np.deg2rad(data["orbital_elements.inclination.value"]))

# --------------------------------------------------------------------------------
fig, ax = plt.subplots(
    2,
    5,
    figsize=fs.figsize(1, aspect=3),
    # sharex=True,
    # sharey=True,
    gridspec_kw={
        "wspace": 0.02,
        "hspace": 0.02,
        "left": 0.065,
        "right": 0.965,
        "top": 0.98,
        "bottom": 0.13,
    },
)
m = maskFINK

# --------------------------------------------------------------------------------
# Distributions
step = 0.04
count = 0
histos = np.zeros( (2,5,45) )
ks_pval = np.zeros( (2,5) )
for i in range(0, 2):
    for j in range(0, 5):
        if i==1 and j==4:
            break

        # Select sample according to inclination
        mi = (Is[m] >= count * step) * (Is[m] < (count + 1) * step)
        nn,bb,_ = ax[i, j].hist(data["lon_rand"][m][mi], bins=45, density=True, histtype="step")
        histos[i,j,:] = nn

        # Cumulative
        ax[1,4].hist(data["lon_rand"][m][mi], bins=45, density=True, cumulative=True, histtype="step", color='C0', alpha=0.35)

        # Axes
        ax[i, j].set_ylim(0, 5.9e-3)
        ax[i, j].set_xlim(0, 360)

        # KS test
        if i==0 and j==0:
            ref = nn

        res = ks_2samp( ref, histos[i,j,:])
        print(f"{i:1d} {j:1d}  {len(data[m][mi]):5d}  {res.statistic:5.3f} {res.pvalue:5.3f}") 


        if i==0:
            yTit, yNum, yFrac = 0.5e-3, 5e-3, 5e-3
        else:
            yTit, yNum, yFrac = 5e-3, 0.5e-3, 0.5e-3
        
        ax[i, j].text(
            180,
            yTit,
            r"{:.2f} $\leq \sin(i) <$ {:.2f}".format(count * step, (count + 1) * step),
            ha="center",
            fontsize="x-small",
        )
        ax[i, j].text(
            20,
            yNum,
            "{:,d}".format(len(data["lon_rand"][m][mi])),
            ha="left",
            fontsize="x-small",
        )
        ax[i, j].text(
            340,
            yFrac,
            "({:.2f}%)".format(
                len(data["lon_rand"][m][mi]) / len(data["lon_rand"][m]) * 100
            ),
            ha="right",
            fontsize="x-small",
        )

        count += 1
        

# --------------------------------------------------------------------------------
# Axes
ax[1, 2].set_xlabel(r"Longitude of the spin ($^{\circ}$)")

for a in ax[:, 0]:
    a.set_ylabel(r"Density")

for a in ax.ravel():
    a.set_xticks([0, 90, 180, 270, 360])
    a.set_yticks([0, 0.002, 0.004 ])


for a in ax[:, 1:].ravel():
    a.set_yticklabels([])

for a in ax[0:, 0:].ravel():
    a.set_xticklabels(['', '', '', '', ''])

for a in ax[1, :-1]:
    a.set_xticklabels([0, 90, 180, 270, " "])
ax[1, -1].set_xticklabels([0, 90, 180, 270, 360])

ax[1,-1].yaxis.tick_right()
ax[1,-1].set_yticks([0, 0.5, 1])
ax[1,-1].set_yticklabels([0, 0.5, 1])
ax[1,-1].text(180,.85,'Cumulative', ha='center',
            fontsize="x-small")
ax[1,-1].text(180,.10,'Comparison', ha='center',
            fontsize="x-small")


# --------------------------------------------------------------------------------
fig.savefig(os.path.join(data_fink, "gfx", "article", "longitude_inclination.pgf"))
fig.savefig(
    os.path.join(data_fink, "gfx", "article", "longitude_inclination.png"),
    facecolor="white",
    dpi=180,
)
plt.close()

0 0   4054  0.000 1.000
0 1  11351  0.133 0.825
0 2  13609  0.133 0.825
0 3   9815  0.156 0.653
0 4   9288  0.111 0.948
1 0   8790  0.222 0.218
1 1   5702  0.178 0.480
1 2   2161  0.156 0.653
1 3   1049  0.156 0.653


# 1-to-1 KS tests

In [ ]:
ks_pval = np.zeros((10,10))

for i in range(0, 10):
    for j in range(0, 10):
    
        # Select sample according to inclination
        mi = (Is[m] >= i * step) * (Is[m] < (i + 1) * step)
        mj = (Is[m] >= j * step) * (Is[m] < (j + 1) * step)
        set_1 = data["lon_rand"][m][mi]
        set_2 = data["lon_rand"][m][mj]

        res = ks_2samp( set_1, set_2 )
        ks_pval[i,j] = res.pvalue
        

In [ ]:
ks_005 = ks_pval.copy()

cond = ks_005 < 0.05
ks_005[cond] = 1
ks_005[~cond] = 0


In [ ]:
fig, ax = plt.subplots(figsize=fs.figsize(0.5, aspect=1))

to_show = ks_pval
# to_show = ks_005
im = ax.imshow(to_show)

# Show all ticks and label them with the respective list entries
ax.set_xticks(np.arange(10))
ax.set_yticks(np.arange(10))

# Loop over data dimensions and create text annotations.
for i in range(10):
    for j in range(10):
        text = ax.text(j, i, f"{100*to_show[i, j]:4.1f}", fontsize='xx-small', 
                       ha="center", va="center", color="w")

fig.savefig(
    os.path.join(data_fink, "gfx", "article", "longitude_inclination_ks.png"),
    facecolor="white",
    dpi=180,
)

plt.close()

# Correlation node vs longitude?

In [26]:
data[ ['orbital_elements.inclination.value', 'orbital_elements.node_longitude.value']].describe()

,orbital_elements.inclination.value,orbital_elements.node_longitude.value
count,116427.000000,116427.000000
mean,9.044755,173.413703
std,6.116269,101.605266
min,0.022011,0.001573
25%,4.467365,88.089558
50%,7.557507,166.870726
75%,12.456671,258.535726
max,125.349560,359.985078


In [51]:
fig, ax = plt.subplots( figsize=fs.figsize(0.5),
                           gridspec_kw={
        "wspace": 0.02,
        "hspace": 0.02,
        "left": 0.12,
        "right": 0.98,
        "top": 0.98,
        "bottom": 0.13,
    },
)

# --------------------------------------------------------------------------------
# Histogram
r_lon = [0,360]
r_nod = [0,360]
b_lon = 60
b_nod = 60
ax.hist2d( data['orbital_elements.node_longitude.value'],
           data["lon_rand"],
           rasterized=True, range=[r_nod, r_lon], bins=[b_nod, b_lon], cmap='Blues')

# --------------------------------------------------------------------------------
# Expected trend from 0-180 obliquity
for i in range(4):
    # Interval of 90 deg
    x = np.linspace(i*90, (i+1)*90-1)

    # If longitude is correct
    ax.plot( x, (x - 90) % 360, color='black' )

    # If longitude is 180 deg off
    ax.plot( x, (x - 90 - 180) % 360, color='black' )




# --------------------------------------------------------------------------------
# Axes
ax.set_xlabel(r"Longitude of the ascending node ($^{\circ}$)")
ax.set_ylabel(r"Longitude of the spin ($^{\circ}$)")


# --------------------------------------------------------------------------------
fig.savefig(
    os.path.join(data_fink, "gfx", "article", "longitude_inclination_test.png"),
    facecolor="white",
    dpi=180,
)

plt.close()

# Old version

In [ ]:
Is = np.sin(np.deg2rad(data["orbital_elements.inclination.value"]))

# --------------------------------------------------------------------------------
fig, ax = plt.subplots(
    2,
    5,
    figsize=fs.figsize(1, aspect=3),
    # sharex=True,
    # sharey=True,
    gridspec_kw={
        "wspace": 0.02,
        "hspace": 0.02,
        "left": 0.1,
        "right": 0.98,
        "top": 0.98,
        "bottom": 0.13,
    },
)
m = maskFINK

# --------------------------------------------------------------------------------
# Distributions
step = 0.04
count = 0
histos = np.zeros( (2,5,45 ))
for i in range(0, 2):
    for j in range(0, 5):
        mi = (Is[m] >= count * step) * (Is[m] < (count + 1) * step)
        nn,bb,_ = ax[i, j].hist(data["lon_rand"][m][mi], bins=45, density=True, histtype="step")
        histos[i,j,:] = nn

        ax[i, j].set_ylim(0, 5.9e-3)
        ax[i, j].set_xlim(0, 360)

#         ax[i, j].text(
#             180,
#             5e-3,
#             r"{:.2f} $\leq \sin(i) <$ {:.2f}".format(count * step, (count + 1) * step),
#             ha="center",
#             fontsize="x-small",
#         )
#         ax[i, j].text(
#             180,
#             1e-3,
#             "{:d} ({:.2f}%)".format(len(data["lon_rand"][m][mi]),
#                 len(data["lon_rand"][m][mi]) / len(data["lon_rand"][m]) * 100
#             ),
#             ha="center",
#             fontsize="x-small",
#         )

        if i==0:
            yTit, yNum, yFrac = 0.5e-3, 5e-3, 5e-3
        else:
            yTit, yNum, yFrac = 5e-3, 0.5e-3, 0.5e-3
        
        ax[i, j].text(
            180,
            yTit,
            r"{:.2f} $\leq \sin(i) <$ {:.2f}".format(count * step, (count + 1) * step),
            ha="center",
            fontsize="x-small",
        )
        ax[i, j].text(
            20,
            yNum,
            "{:,d}".format(len(data["lon_rand"][m][mi])),
            ha="left",
            fontsize="x-small",
        )
        ax[i, j].text(
            340,
            yFrac,
            "({:.2f}%)".format(
                len(data["lon_rand"][m][mi]) / len(data["lon_rand"][m]) * 100
            ),
            ha="right",
            fontsize="x-small",
        )

        count += 1
        

# --------------------------------------------------------------------------------
# Axes
ax[1, 2].set_xlabel(r"Longitude of the spin ($^{\circ}$)")

for a in ax[:, 0]:
    a.set_ylabel(r"Density")

for a in ax[:, 1:].ravel():
    a.set_yticklabels(['', '', '', '', ''])
    
for a in ax[0:, 0:].ravel():
    a.set_xticklabels(['', '', '', '', ''])

for a in ax.ravel():
    a.set_xticks([0, 90, 180, 270, 360])
for a in ax[1, :-1]:
    a.set_xticklabels([0, 90, 180, 270, " "])
ax[1, -1].set_xticklabels([0, 90, 180, 270, 360])

# --------------------------------------------------------------------------------
fig.savefig(os.path.join(data_fink, "gfx", "article", "longitude_inclination_old.pgf"))
fig.savefig(
    os.path.join(data_fink, "gfx", "article", "longitude_inclination_old.png"),
    facecolor="white",
    dpi=180,
)